In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

from itertools import product

In [2]:
ticker = "JPM"
data = yf.download(tickers = ticker, start="2006-01-01", end="2025-01-01")

[*********************100%***********************]  1 of 1 completed


In [3]:
data

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,JPM,JPM,JPM,JPM,JPM,JPM
Date,,,,,,
2006-01-03 00:00:00+00:00,24.241709,40.189999,40.360001,39.299999,39.830002,12838600
2006-01-04 00:00:00+00:00,24.101810,39.619999,40.139999,39.419998,39.779999,13491500
2006-01-05 00:00:00+00:00,24.174805,39.740002,39.810001,39.500000,39.610001,8109400
2006-01-06 00:00:00+00:00,24.345125,40.020000,40.240002,39.549999,39.919998,7966900
2006-01-09 00:00:00+00:00,24.740530,40.669998,40.720001,39.880001,39.880001,16575200
...,...,...,...,...,...,...
2024-12-24 00:00:00+00:00,241.064987,242.309998,242.490005,239.070007,239.429993,3729100
2024-12-26 00:00:00+00:00,241.890717,243.139999,243.259995,240.789993,241.429993,4451800


In [4]:
data["Daily return"] = data["Adj Close"].pct_change() #periods=n
data["State"] = np.where(data["Daily return"] >= 0, "up", "down")

In [5]:
up_counts = len(data[data["State"] == "up"])
down_counts = len(data[data["State"] == "down"])

up_to_up = len(data[(data["State"] == "up") & (data["State"].shift(-1) == "up")])/up_counts #/len(data.query('State=="up"'))
up_to_down = len(data[(data["State"] == "up") & (data["State"].shift(-1) == "down")])/len(data[data["State"] == "up"])
down_to_up = len(data[(data["State"] == "down") & (data["State"].shift(-1) == "up")])/len(data[data["State"] == "down"])
down_to_down = len(data[(data["State"] == "down") & (data["State"].shift(-1) == "down")])/down_counts

In [6]:
transition_matrix = pd.DataFrame({
    "up": [up_to_up, down_to_up],
    "down": [up_to_down, down_to_down]
}, index=["up", "down"])

In [7]:
print(transition_matrix)

            up      down
up    0.495543  0.504052
down  0.538262  0.461738


In [8]:
condition1 = data[
    (data["State"] == "up") 
    & (data["State"].shift(1) == "down")
    & (data["State"].shift(2) == "down")
    & (data["State"].shift(3) == "down")
    & (data["State"].shift(4) == "down")
    & (data["State"].shift(5) == "down")
    ]

condition2 = data[
    (data["State"] == "down")
    & (data["State"].shift(1) == "down")
    & (data["State"].shift(2) == "down")
    & (data["State"].shift(3) == "down")
    & (data["State"].shift(4) == "down")
    ]

In [9]:
print(f"P(up | 5 down) = {len(condition1)/ len(condition2)}")

P(up | 5 down) = 0.6262626262626263


In [10]:
test = data
test["test"] = data["State"].shift(1)
test["test2"] = data["State"].shift(2)
test

Price,Adj Close,Close,High,Low,Open,Volume,Daily return,State,test,test2
Ticker,JPM,JPM,JPM,JPM,JPM,JPM,,,,
Date,,,,,,,,,,
2006-01-03 00:00:00+00:00,24.241709,40.189999,40.360001,39.299999,39.830002,12838600,NaN,down,None,None
2006-01-04 00:00:00+00:00,24.101810,39.619999,40.139999,39.419998,39.779999,13491500,-0.005771,down,down,None
2006-01-05 00:00:00+00:00,24.174805,39.740002,39.810001,39.500000,39.610001,8109400,0.003029,up,down,down
2006-01-06 00:00:00+00:00,24.345125,40.020000,40.240002,39.549999,39.919998,7966900,0.007045,up,up,down
2006-01-09 00:00:00+00:00,24.740530,40.669998,40.720001,39.880001,39.880001,16575200,0.016242,up,up,up
...,...,...,...,...,...,...,...,...,...,...
2024-12-24 00:00:00+00:00,241.064987,242.309998,242.490005,239.070007,239.429993,3729100,0.016444,up,up,up
2024-12-26 00:00:00+00:00,241.890717,243.139999,243.259995,240.789993,241.429993,4451800,0.003425,up,up,up


In [15]:
sequence = ["down", "down", "down", "down", "down", "up"]

# Build the condition using a loop
condition = True
for i, state in enumerate(sequence):
    condition &= (data["State"].shift(-i) == state)

# Filter the DataFrame
filtered_data = data[condition]

In [29]:
states = ["up", "down"]
transition_probabilities = {}

for length in range(2, 11): 
    for state_comb in product(states, repeat=length):
        condition1 = True
        condition2 = True
        for i, state in enumerate(state_comb):
            # In our case it does not matter if we look forward (i.e. -i) or backwards (i.e. i) because in any case we iterate over all combinations.
            condition1 &= (data["State"].shift(i) == state)
            # The first state is the one we "predict"
            if i != 0:
                condition2 &= (data["State"].shift(i) == state)

        transition_probabilities[state_comb] = (len(data[condition1])/len(data[condition2]), len(data[condition2]))

In [41]:
for state_comb, prob in transition_probabilities.items():
    if prob[0] > 0.7 and prob[1] > 30:
        print(f"Probability of {state_comb[0]} after {state_comb[1:]} is {prob[0]}, pattern appeared {prob[1]} times")

Probability of up after ('down', 'up', 'up', 'up', 'down', 'down', 'down') is 0.7096774193548387, pattern appeared 31 times
Probability of down after ('up', 'down', 'down', 'up', 'up', 'down', 'up') is 0.7941176470588235, pattern appeared 34 times


Exit pattern